![](./docs/images/itam_logo.png)

M. Sc. Liliana Millán Núñez liliana.millan@itam.mx

Marzo 2020

## SparkML

### Agenda 

+ SparkML
    + Pipelines
    + Feature engineering
    + Clasificación y regresión
    + Agrupación
    + Tuneo de hiperparámetros
+ Ejemplo

### Spark ML

`spark.ml` es el módulo de *machine learning* de Spark, diseñado para realizar *machine learning* dentro de spark de manera escalable, sencilla y aprovechando el procesamiento en paralelo.

**Características** 

+ Tiene algoritmos de ML ya implementados con la modificaciones necesarias para aprovechar el ambiente distribuido donde vive Spark: clasificación, regresión, agrupación, filtros colaborativos, etc. 
+ Tiene implementaciones de funciones que ocupamos para hacer *feature engineering*: *feature extraction*, *feature selection*, transformaciones, reducción de dimensionalidad.
+ Permite generar *pipelines* en Spark al estilo de los pipelines de `scikitlearn`.
+ Tiene una libería de utilerías con álgebra lineal, estadística, manejo de datos, etc.

¿Por qué hay un `spark.mllib` y un `spark.ml`? 

En la primer versión de Spark no existía la abstracción de *DataFrame* -el *wrapper* de los RDD- y todos los algoritmos de ML desarrollados en Spark interactuaban directamente con el RDD, todas estas implementaciones se encuentran en el paquete `spark.mllib` -que ya está descontinuada-. Una vez que salió la versión 2 de Spark y con ella los nuevos objetos *SparkSession* y *DataFrame* los algoritmos de ML fueron modificados -algunos- para que solo tengan interacción con la abstracción *DataFrame* y con ello surgió la librería `spark.ml` que es la que utilizaremos nosotros. Aún no están todos los algoritmos de `spark.mllib` implementados en *DataFrame*, en la versión 2.4.5 de Spark, la librería de `spark.mllib` ya está en estatus de solo mantenimiento para que a partir de Spark 3.0 la librería será removida completamente de Spark y solo ocupar la interacción con los *DataFrames*. 

*Anyway* Para confundir más a la banda, el nombre oficial de la herramienta que ocupa Spark para ML se conoce como **MLlib** (╯°□°)╯︵ ┻━┻ aunque realmente se refieren a la librería `spark.ml`.

[Spark ML API](https://spark.apache.org/docs/latest/ml-guide.html)

#### Pipelines 

El diseño de los *pipelines* de Spark está inspirado en los *pipelines* de `scikit-learn`. Un *pipeline* en Spark está formado por los siguientes elementos: 

+ **DataFrame:** API que ocupa los *DataFrame* de SparkSQL para poder agregar otros tipos de datos que pueden ser útiles para ML -*vector*-

+ **Transformer:** Algoritmo que transforma un *DataFrame* en otro DataFrame, recuerda que los *DataFrame* en Spark envuelven a un RDD y un RDD no puede ser modificado!. Para hacer una transformación se ocupa el método `transform()`. Los casos en los que ocuparemos un `transform` pueden ser agregar una nueva columna -por ejemplo *feature engineering*-, o por ejemplo una vez que se ha pasado un modelo de aprendizaje poner la respuesta final del modelo como parte del *DataFrame* original -etiqueta, score-. 

+ **Estimator:** Algoritmos que se aplican a un *DataFrame* para producir un *Transformer*. Los estimators son los que ocupan el método `fit()` para poder realizar un entrenamiento. El método `fit` recibe como parámetro un *DataFrame* y devuelve un modelo -que es un *transformer*-. Por ejemplo: Un algoritmo de regresión lineal es un *estimator* que tiene su método `fit` a través del cual entrena el algoritmo. 


$\rightarrow$ Es importante conocer que por cada instancia de un *transformer* o *estimator* se genera un ID a través del cuál es reconocido durante todo el *pipeline* y por lo tanto podemos llamarlo más adelante en el pipeline.

+ **Pipeline:** Es una secuencia de procesos/etapas generado por *transformers* y *estimators* para hacer un *workflow* de ML. Cada *transformer*/*estimator* es una etapa dentro de la secuencia del *pipeline*, cada paso se corre en el orden establecido y el *DataFrame* de entrada es transformado por cada paso, si el paso es un *transformer* entonces se le aplica el método `transform` y si el paso es un `estimator` se le aplica el método `fit`. 


![](./docs/images/spark_estimator_transformer.png)
<br>

Por ejemplo: Si tuviéramos un texto al cuál quisieramos aplicarle un análisis de sentimiento, el *pipeline* podría consistir en los siguientes pasos: 

Suposiciones: 
+ Tenemos un corpus.
+ Tenemos las palabras asociadas a un sentimiento.

+ Separar cada documento en palabras.
+ Convertir cada palabra de cada documento en un vector numérico.
+ Utilizando el vector numérico y las etiquetas asociadas -del sentimiento- ocupar un modelo de clasificación 

![](./docs/images/spark_pipelines.png)

<br>
\* Fuente: [Spark ML Guide](https://spark.apache.org/docs/latest/ml-guide.html)


![](./docs/images/pointer.png) En Spark, un pipeline **es** un *estimator* (al igual que en `sklearn`, por lo que puede hacer llamada al método `fit`, al hacer esto se genera un *PipelineModel* -que es un *transformer*-. Cuando querramos ocupar modelos entrenados para producción deberemos ocupar el *PipelineModel* generado en el momento de entranamiento al hacer una llamada a su método *transform*, de esta manera todos los *estimators* del *pipeline* original son convertidos a *transformer* asegurándonos de que en pruebas tendremos los mismos pasos/trasnformaciones ocupados para el entrenamiento del modelo. ╭(◔ ◡ ◔)/

![](./docs/images/spark_pipeline_model.png)
<br>
\*Fuente: [Spark ML Guide](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml) 

Un *Pipeline* en Spark está representado como un DAG, el ejemplo anterior es un DAG lineal, pero no necesariamente deben ser lineales, basta con que cumplan las características de ser un DAG -grafo **acíclico** dirigido-. Es por esta razón que cada instanciación de un *transformer* o *estimator* tiene asociado un ID y debe ser único, si necesitaramos un mismo *transformer* en el *pipeline* requerimos de generar otro *transformer* -aunque tenga el mismo código- :( (ya sé! esto medio que le da en la ma al principio de *reuse* pero ... por el momento así se resuelve en Spark en pro de tener un *pipeline*), Spark revisa en tiempo de ejecución que no se rompa "algo" antes de correr el *pipeline* -*lazy*-

+ **Parameter:** API con la que se pueden compartir parámetros entre *Estimators* y *Transformers*. Ocupamos el objeto `Param` que es un parámetro nombrado con documentación auto contenida en un `ParamMap` -diccionario de parámetro, valor-.

En Spark hay dos maneras de pasar parámetros a los algoritmos de ML:

1. Configurar los parámetros fijos de los algoritmos a ocupar (*setters*)
2. Pasar un `ParamMap` con los parámetros y sus valores a través de `fit` o `transform`, si se envían parámetros de esta manera se hace *override* a los específicados vía *setters*

Lo lindo de estos objetos es que cada definición dentro del `ParamMap` es "atado" a un *estimator* o *transformer* en específico -a través del ID antes mencionado-. Por ejemplo: si tuvieramos en un *pipeline* dos regresiones logísticas -`lr1` y `lr2`- podríamos ocupar un `ParamMap` que establezca el valor de las iteraciones máximas de cada regresión: 


In [ ]:
#pyspark 
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# En Spark se puede crear un DataFrame de un RDD, de una lista o de un DataFrame de Pandas, 
# aquí lo estamos creando con una lista que contiene tuplas de (label, features)
# tal cual lo hacíamos en sklearn, y le estamos agregando los nombres de cada columna.
# Vectors.dense recibe una lista como parámetro
# Este DataFrame lo estamos ocupando como nuestro set de entrenamiento mock!
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# Como lo hacíamos en sklearn, primero instanciamos el modelo que quremos
# ocupar con los parámetros que nosotros queremos tener para este 
# modelo en particular --configuramos el modelo--
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Veamos la documentacion del modelo y que parametros le pusimos a nuestra
# configuracion
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Ocupemos el modelo que configuramos para entrenar con lo datos que
# creamos en el DataFrame training
model_1 = lr.fit(training)

# model_1 es un transfomer creado a traves de un estimador (LogisticRegression)
print("Model 1 was fit using parameters: ")
# aqui estamos obteniendo la configuracion con la que se entreno
# la regresion logistica que ocupamos
print(lr.extractParamMap())


# Tambien podemos especificar los parametros con los que queremos que 
# corra el modelo utilizando el diccionario de ParamMap
# Creamos un diccionario -se puede llamar como quieras!- que tenga
# como llave el nombre del parametro que quieres modificar, con el valor
# correspondiente.
param_map = {lr.maxIter: 20}
# Si el valor ya existe en el diccionario puedes actualizarlo
param_map[lr.maxIter] = 30  
# Tambien puedes actualizar varios parametros del diccionario al mismo tiempo
param_map.update({lr.regParam: 0.1, lr.threshold: 0.55}) 

# Se pueden combinar diferentes diccionarios...realmente puedes tener
# un solo diccionario con los parametros de diversos modelos que ocupes en el
# pipeline sin ningun problema, pues el valor asociado es por objeto (ID)
# aqui estamos cambiando el nombre de la columna que guarda la salida del
# modelo, por default se llama 'probability' -> verificar documentacion del 
# metodo
param_map_2 = {lr.probabilityCol: "my_probability"}  
param_map_combined = param_map.copy()
param_map_combined.update(param_map_2)
#puedes ver el contenido del diccionario con param_map_combined.items() -> python 3.5.2

# Entrenemos una segunda regresion logistica con los nuevos parametros que 
# establecimos a traves del paramMap
# En este fit estamos enviando tanto los datos como los parametros a ocupar en el
# modelo de regresion logistica
model_2 = lr.fit(training, param_map_combined)
print("Model 2 was fit using parameters: ")
# aqui queremos ver cono que parametros se quedo configurado el modelo
# que ocupamos para entrenar
print(lr.extractParamMap())

# Creemos el data frame que tendra los datos de prueba mock!
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

# Ahora sí, hagamos predicciones sobre el *set* de pruebas utilizando 
# el método `transform` (aquí no hay `predict`!)
prediction = model_2.transform(test)
# la respuesta es un DataFrame (la salida de un transform en su DataFrame)
# por lo que podemos aplicarle los metodos de SparkSQL :)
# verificamos que si es un DataFrame...
type(prediction)
# veamos que columnas tiene este DataFrame (como el names de R)
prediction.columns
# Aqui estamos seleccionando las columnas features, label, 
# my_probability -> que es el nombre que nosotros especificamos anteriormente en
# ParamMap, y la columna prediction que es el nombre por default que regresa
# el modelo al parametro 'predictionCol' -> ver documentacion
# el collect hara que se regresen los resultados al drive!!! 
result = prediction.select("features", "label", "my_probability", "prediction") \
    .collect()


for row in result:
    print("features={}, label={} -> prob={}, prediction={}".format( \
    row.features, row.label, row.my_probability, row.prediction))